In [1]:
import numpy as np
import pandas as pd
import os

## Data collection

In [2]:
# Design family and directory
fam = '03'
#base_dir = r'C:\FBr\Weiterbildung\Project'
base_dir = r'D:\Entwicklungen\share\DataScienceProject'
#base_dir = r'D:\FBr\DataScienceProject'

In [3]:
# Import of parameters and design points
all_data = {}
fam_dir = os.path.join(base_dir, fam, 'data-' + fam)
ft_param_path = os.path.join(base_dir, 'Scripts', str(fam) + '-coll-parameters.ft')
all_data[fam] = {}

# Column names and widths
ident_cols = ['dp_no', 'mode_no', 'freq', 'rel_dev', 'mean_in_disp', 'mean_out_disp', 'mode_gain', 'mode_mac']
mode_cols = ['mode_no', 'freq', 'rel_dev', 'mean_in_disp', 'mean_out_disp', 'mode_gain', 'mode_mac']
node_cols = ['node_no', 'x_coord', 'y_coord', 'z_coord']
def_cols = ['mode' + str(x) for x in range(1, 101)]
ident_wd = [8, 12, 15, 15, 15, 15, 15, 15]
mode_wd = [8, 15, 15, 15, 15, 15, 15]
node_wd = [10, 20, 20, 20]
def_wd = [20 for x in range(100)]

# Subset of modes for each design family
fam_mode_subset = {'01': range(1, 14), '02': range(17, 31), '03': range(25, 101)}
lst_modes = ['mode' + str(x) for x in fam_mode_subset[fam]]

# Geometrical parameters  
all_data[fam]['parameters'] = {}
params_file = os.path.join(fam_dir, 'design-points-' + fam + '.csv')
df_param = pd.read_csv(params_file, sep=';')

# Identification of the longitudinal mode
ident_file = os.path.join(fam_dir, 'mode-ident-full-' + fam + '.dat')
df_ident = pd.read_fwf(ident_file, widths=ident_wd, comment='#', names=ident_cols)

# Merge both DataFrames
all_data[fam]['parameters'] = pd.merge(df_param, df_ident, how='outer', on='dp_no')

# List of design points
lst_dp = df_param['dp_no'].to_list()

# Export DataFrame as binary file
all_data[fam]['parameters'].to_feather(ft_param_path)

In [4]:
all_data[fam]['parameters']

,dp_no,dim_x,dim_y,dim_z,nb_slots_x,slot_x_length,slot_x_distance,nb_slots_z,slot_z_length,slot_z_distance,...,cut_z_start,cut_z_end,cut_z_depth,mode_no,freq,rel_dev,mean_in_disp,mean_out_disp,mode_gain,mode_mac
0,1000,175.00,120.00,240.00,4,77.000,48.000,2,77.000,58.000,...,25.000,55.000,0.10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,175.06,121.95,240.31,4,75.993,43.235,2,78.219,59.508,...,25.293,66.060,1.99450,77.0,19217.965,0.92217,-0.371,0.280,0.75609,0.921784
2,1002,173.45,122.77,238.13,4,81.168,45.859,2,78.924,57.208,...,26.997,56.481,4.80300,79.0,19386.656,0.95697,-0.386,0.293,0.75940,0.923429
3,1003,175.85,116.71,238.10,4,77.532,45.260,2,80.099,49.215,...,22.264,51.204,0.81847,79.0,20087.459,0.51795,-0.337,0.321,0.95450,0.971387
4,1004,173.51,116.37,238.30,4,82.428,51.007,2,74.757,50.840,...,37.379,58.304,1.02820,80.0,20209.177,0.79699,-0.351,0.319,0.90825,0.948260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8074,11995,175.18,119.50,239.53,2,78.314,80.925,2,77.038,54.535,...,30.256,66.937,4.66870,55.0,20351.555,1.20382,-0.309,0.295,0.95621,0.913093
8075,11996,176.27,117.29,240.59,2,77.933,71.773,2,79.789,63.279,...,33.988,64.708,4.83750,58.0,20703.179,1.93926,0.355,-0.259,0.73108,0.764613
8076,11997,174.01,117.81,239.26,2,80.402,81.905,2,79.398,60.146,...,39.064,60.663,1.04200,56.0,20621.116,1.13667,-0.322,0.304,0.94193,0.919224
8077,11998,174.61,119.50,239.41,2,74.848,74.107,2,74.870,57.402,...,35.845,56.585,4.99810,55.0,20058.828,4.79138,-0.112,0.218,1.94744,0.328624


In [5]:
# Import of modes, nodes and displacements for each design point
all_data[fam]['modes'] = pd.DataFrame()
all_data[fam]['nodes'] = pd.DataFrame()
all_data[fam]['defs'] = pd.DataFrame()
ft_mode_path = os.path.join(base_dir, 'Scripts', str(fam) + '-coll-modes.ft')
ft_node_path = os.path.join(base_dir, 'Scripts', str(fam) + '-coll-nodes.ft')
ft_defs_path = os.path.join(base_dir, 'Scripts', str(fam) + '-coll-defs.ft')

#for dp in [1000, 1001, 1002]:
for dp in lst_dp:
        
    print ('. Design point', str(dp), 'of', str(lst_dp[-1]))

    # Mode file
    mode_file = 'tbl-modes-' + str(dp) + '.dat'
    mode_path = os.path.join(fam_dir, 'modes', mode_file)

    if os.path.exists(mode_path):

        df_nodes = pd.read_fwf(mode_path, widths=mode_wd, comment='#', names=mode_cols)

        # Add design point as column
        n_modes = len(df_nodes)   # Number of modes
        df_nodes.insert(0, 'dp_no', pd.Series(dp*np.ones(n_modes), index=df_nodes.index))
        
        # Add subset of modes to dictionary
        all_data[fam]['modes'] = pd.concat([all_data[fam]['modes'], df_nodes[df_nodes['mode_no'].isin(fam_mode_subset[fam])]], axis=0, ignore_index=True)


    # Node and displacement files
    node_file = 'tbl-nodes-' + str(dp) + '.dat'
    node_path = os.path.join(fam_dir, 'nodes', node_file)
    def_file = 'tbl-def-y-' + str(dp) + '.dat'
    def_path = os.path.join(fam_dir, 'defs', def_file)

    if (os.path.exists(node_path) & os.path.exists(def_path)):

        # ---
        # Node file
        df_nodes = pd.read_fwf(node_path, widths=node_wd, comment='#', names=node_cols)

        # Write design point as foreign key
        node_list = df_nodes['node_no'].astype(int).to_list()
        df_nodes = pd.DataFrame({'dp_no': dp}, index=df_nodes.index).join(df_nodes)

        # Add subset of modes to dictionary
        all_data[fam]['nodes'] = pd.concat([all_data[fam]['nodes'], df_nodes], axis=0, ignore_index=True)

        # ---
        # Displacement file
        df_defs = pd.read_fwf(def_path, widths=def_wd, header=None, names=def_cols)

        # Keep the displacements of the subset of modes only
        df_defs = df_defs[lst_modes]

        # Write displacements and add "node_no", "dp_no" as foreign keys
        df_defs = pd.DataFrame({'node_no': node_list}, index=df_defs.index).join(df_defs)
        df_defs = pd.DataFrame({'dp_no': dp}, index=df_defs.index).join(df_defs)

        # Add displacements to dictionary
        all_data[fam]['defs'] = pd.concat([all_data[fam]['defs'], df_defs], axis=0, ignore_index=True)


# Specify data types
all_data[fam]['modes'] = all_data[fam]['modes'].astype({'dp_no': 'int', 'mode_no': 'int'})
all_data[fam]['modes']['rel_dev'] = pd.to_numeric(all_data[fam]['modes']['rel_dev'], errors='coerce')
all_data[fam]['nodes'] = all_data[fam]['nodes'].astype({'dp_no': 'int', 'node_no': 'int'})

# Set node coordinates in milimeters
all_data[fam]['nodes'][['x_coord', 'y_coord', 'z_coord']] = all_data[fam]['nodes'][['x_coord', 'y_coord', 'z_coord']]*1e3

# Export DataFrames as binary file
all_data[fam]['modes'].to_feather(ft_mode_path)
all_data[fam]['nodes'].to_feather(ft_node_path)
all_data[fam]['defs'].to_feather(ft_defs_path)

. Design point 1000 of 11999
. Design point 1001 of 11999
. Design point 1002 of 11999
. Design point 1003 of 11999
. Design point 1004 of 11999
. Design point 1005 of 11999
. Design point 1006 of 11999
. Design point 1007 of 11999
. Design point 1008 of 11999
. Design point 1009 of 11999
. Design point 1010 of 11999
. Design point 1011 of 11999
. Design point 1012 of 11999
. Design point 1013 of 11999
. Design point 1014 of 11999
. Design point 1015 of 11999
. Design point 1016 of 11999
. Design point 1017 of 11999
. Design point 1018 of 11999
. Design point 1019 of 11999
. Design point 1020 of 11999
. Design point 1021 of 11999
. Design point 1022 of 11999
. Design point 1023 of 11999
. Design point 1024 of 11999
. Design point 1025 of 11999
. Design point 1026 of 11999
. Design point 1027 of 11999
. Design point 1028 of 11999
. Design point 1029 of 11999
. Design point 1030 of 11999
. Design point 1031 of 11999
. Design point 1032 of 11999
. Design point 1033 of 11999
. Design point

In [6]:
all_data[fam]['modes']

,dp_no,mode_no,freq,rel_dev,mean_in_disp,mean_out_disp,mode_gain,mode_mac
0,1000,25,11705.549,120.61732,-0.121,0.002,0.01985,0.076401
1,1000,26,11987.927,412.69239,-0.009,0.002,0.26013,0.000220
2,1000,27,12187.892,474.99649,0.000,0.001,2.26471,0.000035
3,1000,28,12318.561,11.80434,-0.065,-0.046,0.71102,0.009971
4,1000,29,12598.709,847.60025,0.001,0.000,0.92230,0.000006
...,...,...,...,...,...,...,...,...
607463,11999,96,27491.449,967.39541,0.001,-0.001,1.31907,0.000015
607464,11999,97,27520.148,245.47758,-0.001,-0.005,4.99439,0.000167
607465,11999,98,27581.676,422.81995,-0.002,0.004,1.66456,0.000128
607466,11999,99,27587.681,100.51100,0.008,0.017,2.04339,0.001967


In [7]:
all_data[fam]['nodes']

,dp_no,node_no,x_coord,y_coord,z_coord
0,1000,1,-120.015000,0.0,-76.720000
1,1000,11,-120.015000,0.0,76.720000
2,1000,12,-120.015000,0.0,57.540000
3,1000,13,-120.015000,0.0,38.360000
4,1000,14,-120.015000,0.0,19.180000
...,...,...,...,...,...
3960055,11999,10463,61.672218,0.0,17.464598
3960056,11999,10464,79.323372,0.0,11.214439
3960057,11999,10487,45.118361,0.0,11.704285
3960058,11999,10506,75.736356,0.0,-29.317255


In [9]:
all_data[fam]['defs']

,dp_no,node_no,mode25,mode26,mode27,mode28,mode29,mode30,mode31,mode32,...,mode91,mode92,mode93,mode94,mode95,mode96,mode97,mode98,mode99,mode100
0,1000,1,0.134142,0.216114,0.193092,-0.297784,0.046671,0.101679,-0.038922,0.039048,...,-0.290045,-0.330238,-0.418750,-0.396922,0.167523,-0.294506,0.305611,0.242859,-0.131780,-0.102837
1,1000,11,0.133235,-0.224205,0.177632,-0.296075,-0.053283,-0.110080,-0.023488,0.037248,...,-0.295936,-0.325200,-0.413388,-0.399802,0.185901,0.240496,0.349572,0.235923,0.139851,-0.066484
2,1000,12,0.067980,-0.244073,0.067255,-0.170133,0.017068,-0.028849,-0.032300,-0.002932,...,0.002639,-0.088270,-0.416453,-0.475318,0.080366,0.157874,-0.031110,0.535567,-0.200790,-0.409373
3,1000,13,-0.013566,-0.263616,-0.039998,-0.027363,0.124321,0.080527,-0.013113,-0.022632,...,0.169028,0.109970,0.005401,-0.042836,-0.001065,0.016996,-0.298478,0.215014,-0.454417,-0.438804
4,1000,14,-0.091604,-0.232150,-0.146628,0.080373,0.196288,0.157715,-0.005128,-0.053346,...,0.000723,0.090866,0.728555,0.792384,-0.042708,-0.064504,-0.377651,-0.633504,-0.411714,-0.202440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3960055,11999,10463,-0.048392,0.036274,0.020404,-0.088032,0.035213,-0.071034,0.067050,-0.045100,...,-0.146413,-0.018394,0.147541,0.098663,-0.081684,0.124870,-0.087999,0.102367,-0.105086,0.046667
3960056,11999,10464,-0.032425,0.054994,0.007382,-0.005498,0.003142,-0.191339,-0.002823,-0.108213,...,-0.120918,-0.181788,0.103812,-0.004514,0.023668,0.056033,-0.066901,0.127630,-0.009349,-0.064460
3960057,11999,10487,-0.101641,-0.017423,0.007340,-0.169164,0.021515,0.072380,0.100115,-0.023856,...,-0.022090,0.094887,0.080802,0.128938,-0.073307,-0.007754,-0.059983,0.085003,0.046614,-0.041904
3960058,11999,10506,-0.038694,-0.091874,-0.020720,-0.017423,-0.021544,-0.079995,0.020790,-0.038876,...,0.297812,-0.063521,-0.124353,-0.047298,-0.063314,0.202607,-0.001829,-0.277119,0.012272,0.220565
